In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras import regularizers
from keras.engine.topology import Layer
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import load_data
import matplotlib
matplotlib.use("Agg")
from matplotlib import pyplot as plt
import itertools

###################################################################################################

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.1f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# Models to be passed to Music_Genre_CNN

song_labels = ["Blues","Classical","Country","Disco","Hip hop","Jazz","Metal","Pop","Reggae","Rock"]

###################################################################################################

def metric(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=1), K.argmax(y_pred, axis=1)))

def cnn(num_genres=10, input_shape=(64,173,1)):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(4, 4),
                     activation='relu', #kernel_regularizer=regularizers.l2(0.04),
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 4)))
    model.add(Conv2D(64, (3, 5), activation='relu'
                    , kernel_regularizer=regularizers.l2(0.04)
                    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (2, 2), activation='relu'
       # , kernel_regularizer=regularizers.l2(0.04)
        ))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.04)))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.04)))
    model.add(Dense(num_genres, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                  metrics=[metric])
    return(model)

###################################################################################################

# Main network thingy to train

class model(object):

    def __init__(self, ann_model):
        self.model = ann_model()

    def train_model(self, train_x, train_y,
                val_x=None, val_y=None,
                small_batch_size=200, max_iteration=300, print_interval=1,
                test_x=None, test_y=None):

        m = len(train_x)

        for it in range(max_iteration):

            # split training data into even batches
            batch_idx = np.random.permutation(m)
            train_x = train_x[batch_idx]
            train_y = train_y[batch_idx]

            num_batches = int(m / small_batch_size)
            for batch in range(num_batches):

                x_batch = train_x[ batch*small_batch_size : (batch+1)*small_batch_size]
                y_batch = train_y[ batch*small_batch_size : (batch+1)*small_batch_size]
                print("starting batch\t", batch, "\t Epoch:\t", it)
                self.model.train_on_batch(x_batch, y_batch)

            if it % print_interval == 0:
                validation_accuracy = self.model.evaluate(val_x, val_y)
                training_accuracy = self.model.evaluate(train_x, train_y)
                testing_accuracy = self.model.evaluate(test_x, test_y)
                # print of test error used only after development of the model
                print("\nTraining accuracy: %f\t Validation accuracy: %f\t Testing Accuracy: %f" %
                      (training_accuracy[1], validation_accuracy[1], testing_accuracy[1]))
                print("\nTraining loss: %f    \t Validation loss: %f    \t Testing Loss: %f \n" %
                      (training_accuracy[0], validation_accuracy[0], testing_accuracy[0]))
                print( )

            if (validation_accuracy[1] > .81):
                print("Saving confusion data...")
                model_name = "model" + str(100*validation_accuracy[1]) + str(100*testing_accuracy[1]) + ".h5"
                self.model.save(model_name) 
                pred = self.model.predict_classes(test_x, verbose=1)
                cnf_matrix = confusion_matrix(np.argmax(test_y, axis=1), pred)
                np.set_printoptions(precision=2)
                plt.figure()
                plot_confusion_matrix(cnf_matrix, classes=song_labels, normalize=True, title='Normalized confusion matrix')
                print(precision_recall_fscore_support(np.argmax(test_y, axis=1),pred, average='macro')) 
                plt.savefig(str(batch))

###################################################################################################

def main():

#################################################

# Data stuff

    data = load_data.loadall('melspects.npz')

    x_tr = data['x_tr']
    y_tr = data['y_tr']
    x_te = data['x_te']
    y_te = data['y_te']
    x_cv = data['x_cv']
    y_cv = data['y_cv']

    tr_idx = np.random.permutation(len(x_tr))
    te_idx = np.random.permutation(len(x_te))
    cv_idx = np.random.permutation(len(x_cv))

    x_tr = x_tr[tr_idx]
    y_tr = y_tr[tr_idx]
    x_te = x_te[te_idx]
    y_te = y_te[te_idx]
    x_cv = x_cv[cv_idx]
    y_cv = y_cv[cv_idx]

    x_tr = x_tr[:,:,:,np.newaxis]
    x_te = x_te[:,:,:,np.newaxis]
    x_cv = x_cv[:,:,:,np.newaxis]

    y_tr = np_utils.to_categorical(y_tr)
    y_te = np_utils.to_categorical(y_te)
    y_cv = np_utils.to_categorical(y_cv)



    ann = model(cnn)
    ann.train_model(x_tr, y_tr, val_x=x_cv, val_y=y_cv, test_x=x_te, test_y=y_te)

if __name__ == '__main__':
    main()










starting batch	 0 	 Epoch:	 0
starting batch	 1 	 Epoch:	 0
starting batch	 2 	 Epoch:	 0
starting batch	 3 	 Epoch:	 0
starting batch	 4 	 Epoch:	 0
starting batch	 5 	 Epoch:	 0
starting batch	 6 	 Epoch:	 0
starting batch	 7 	 Epoch:	 0
starting batch	 8 	 Epoch:	 0
starting batch	 9 	 Epoch:	 0
starting batch	 10 	 Epoch:	 0
starting batch	 11 	 Epoch:	 0
starting batch	 12 	 Epoch:	 0
starting batch	 13 	 Epoch:	 0
starting batch	 14 	 Epoch:	 0
starting batch	 15 	 Epoch:	 0
starting batch	 16 	 Epoch:	 0
starting batch	 17 	 Epoch:	 0
starting batch	 18 	 Epoch:	 0
starting batch	 19 	 Epoch:	 0
starting batch	 20 	 Epoch:	 0
starting batch	 21 	 Epoch:	 0
starting batch	 22 	 Epoch:	 0
starting batch	 23 	 Epoch:	 0
starting batch	 24 	 Epoch:	 0
starting batch	 25 	 Epoch:	 0
starting batch	 26 	 Epoch:	 0
starting batch	 27 	 Epoch:	 0
starting batch	 28 	 Epoch:	 0
starting batch	 29 	 Epoch:	 0
starting batch	 30 	 Epoch:	 0
starting batch	 31 	 Epoch:	 0
starting batch	 32

starting batch	 0 	 Epoch:	 5
starting batch	 1 	 Epoch:	 5
starting batch	 2 	 Epoch:	 5
starting batch	 3 	 Epoch:	 5
starting batch	 4 	 Epoch:	 5
starting batch	 5 	 Epoch:	 5
starting batch	 6 	 Epoch:	 5
starting batch	 7 	 Epoch:	 5
starting batch	 8 	 Epoch:	 5
starting batch	 9 	 Epoch:	 5
starting batch	 10 	 Epoch:	 5
starting batch	 11 	 Epoch:	 5
starting batch	 12 	 Epoch:	 5
starting batch	 13 	 Epoch:	 5
starting batch	 14 	 Epoch:	 5
starting batch	 15 	 Epoch:	 5
starting batch	 16 	 Epoch:	 5
starting batch	 17 	 Epoch:	 5
starting batch	 18 	 Epoch:	 5
starting batch	 19 	 Epoch:	 5
starting batch	 20 	 Epoch:	 5
starting batch	 21 	 Epoch:	 5
starting batch	 22 	 Epoch:	 5
starting batch	 23 	 Epoch:	 5
starting batch	 24 	 Epoch:	 5
starting batch	 25 	 Epoch:	 5
starting batch	 26 	 Epoch:	 5
starting batch	 27 	 Epoch:	 5
starting batch	 28 	 Epoch:	 5
starting batch	 29 	 Epoch:	 5
starting batch	 30 	 Epoch:	 5
starting batch	 31 	 Epoch:	 5
starting batch	 32

starting batch	 0 	 Epoch:	 10
starting batch	 1 	 Epoch:	 10
starting batch	 2 	 Epoch:	 10
starting batch	 3 	 Epoch:	 10
starting batch	 4 	 Epoch:	 10
starting batch	 5 	 Epoch:	 10
starting batch	 6 	 Epoch:	 10
starting batch	 7 	 Epoch:	 10
starting batch	 8 	 Epoch:	 10
starting batch	 9 	 Epoch:	 10
starting batch	 10 	 Epoch:	 10
starting batch	 11 	 Epoch:	 10
starting batch	 12 	 Epoch:	 10
starting batch	 13 	 Epoch:	 10
starting batch	 14 	 Epoch:	 10
starting batch	 15 	 Epoch:	 10
starting batch	 16 	 Epoch:	 10
starting batch	 17 	 Epoch:	 10
starting batch	 18 	 Epoch:	 10
starting batch	 19 	 Epoch:	 10
starting batch	 20 	 Epoch:	 10
starting batch	 21 	 Epoch:	 10
starting batch	 22 	 Epoch:	 10
starting batch	 23 	 Epoch:	 10
starting batch	 24 	 Epoch:	 10
starting batch	 25 	 Epoch:	 10
starting batch	 26 	 Epoch:	 10
starting batch	 27 	 Epoch:	 10
starting batch	 28 	 Epoch:	 10
starting batch	 29 	 Epoch:	 10
starting batch	 30 	 Epoch:	 10
starting batch	 31

4/4 [==============================] - 0s 75ms/step - loss: 1.8082 - metric: 0.4844

Training accuracy: 0.687000	 Validation accuracy: 0.601562	 Testing Accuracy: 0.484375

Training loss: 1.494172    	 Validation loss: 1.517984    	 Testing Loss: 1.808183 


starting batch	 0 	 Epoch:	 15
starting batch	 1 	 Epoch:	 15
starting batch	 2 	 Epoch:	 15
starting batch	 3 	 Epoch:	 15
starting batch	 4 	 Epoch:	 15
starting batch	 5 	 Epoch:	 15
starting batch	 6 	 Epoch:	 15
starting batch	 7 	 Epoch:	 15
starting batch	 8 	 Epoch:	 15
starting batch	 9 	 Epoch:	 15
starting batch	 10 	 Epoch:	 15
starting batch	 11 	 Epoch:	 15
starting batch	 12 	 Epoch:	 15
starting batch	 13 	 Epoch:	 15
starting batch	 14 	 Epoch:	 15
starting batch	 15 	 Epoch:	 15
starting batch	 16 	 Epoch:	 15
starting batch	 17 	 Epoch:	 15
starting batch	 18 	 Epoch:	 15
starting batch	 19 	 Epoch:	 15
starting batch	 20 	 Epoch:	 15
starting batch	 21 	 Epoch:	 15
starting batch	 22 	 Epoch:	 15
starting batch	

starting batch	 0 	 Epoch:	 19
starting batch	 1 	 Epoch:	 19
starting batch	 2 	 Epoch:	 19
starting batch	 3 	 Epoch:	 19
starting batch	 4 	 Epoch:	 19
starting batch	 5 	 Epoch:	 19
starting batch	 6 	 Epoch:	 19
starting batch	 7 	 Epoch:	 19
starting batch	 8 	 Epoch:	 19
starting batch	 9 	 Epoch:	 19
starting batch	 10 	 Epoch:	 19
starting batch	 11 	 Epoch:	 19
starting batch	 12 	 Epoch:	 19
starting batch	 13 	 Epoch:	 19
starting batch	 14 	 Epoch:	 19
starting batch	 15 	 Epoch:	 19
starting batch	 16 	 Epoch:	 19
starting batch	 17 	 Epoch:	 19
starting batch	 18 	 Epoch:	 19
starting batch	 19 	 Epoch:	 19
starting batch	 20 	 Epoch:	 19
starting batch	 21 	 Epoch:	 19
starting batch	 22 	 Epoch:	 19
starting batch	 23 	 Epoch:	 19
starting batch	 24 	 Epoch:	 19
starting batch	 25 	 Epoch:	 19
starting batch	 26 	 Epoch:	 19
starting batch	 27 	 Epoch:	 19
starting batch	 28 	 Epoch:	 19
starting batch	 29 	 Epoch:	 19
starting batch	 30 	 Epoch:	 19
starting batch	 31

...

starting batch	 11 	 Epoch:	 72
starting batch	 12 	 Epoch:	 72
starting batch	 13 	 Epoch:	 72
starting batch	 14 	 Epoch:	 72
starting batch	 15 	 Epoch:	 72
starting batch	 16 	 Epoch:	 72
starting batch	 17 	 Epoch:	 72
starting batch	 18 	 Epoch:	 72
starting batch	 19 	 Epoch:	 72
starting batch	 20 	 Epoch:	 72
starting batch	 21 	 Epoch:	 72
starting batch	 22 	 Epoch:	 72
starting batch	 23 	 Epoch:	 72
starting batch	 24 	 Epoch:	 72
starting batch	 25 	 Epoch:	 72
starting batch	 26 	 Epoch:	 72
starting batch	 27 	 Epoch:	 72
starting batch	 28 	 Epoch:	 72
starting batch	 29 	 Epoch:	 72
starting batch	 30 	 Epoch:	 72
starting batch	 31 	 Epoch:	 72
starting batch	 32 	 Epoch:	 72
starting batch	 33 	 Epoch:	 72
starting batch	 34 	 Epoch:	 72
starting batch	 35 	 Epoch:	 72
starting batch	 36 	 Epoch:	 72
starting batch	 37 	 Epoch:	 72
starting batch	 38 	 Epoch:	 72
starting batch	 39 	 Epoch:	 72
4/4 [==============================] - 0s 86ms/step - loss: 1.3557 - met

starting batch	 1 	 Epoch:	 77
starting batch	 2 	 Epoch:	 77
starting batch	 3 	 Epoch:	 77
starting batch	 4 	 Epoch:	 77
starting batch	 5 	 Epoch:	 77
starting batch	 6 	 Epoch:	 77
starting batch	 7 	 Epoch:	 77
starting batch	 8 	 Epoch:	 77
starting batch	 9 	 Epoch:	 77
starting batch	 10 	 Epoch:	 77
starting batch	 11 	 Epoch:	 77
starting batch	 12 	 Epoch:	 77
starting batch	 13 	 Epoch:	 77
starting batch	 14 	 Epoch:	 77
starting batch	 15 	 Epoch:	 77
starting batch	 16 	 Epoch:	 77
starting batch	 17 	 Epoch:	 77
starting batch	 18 	 Epoch:	 77
starting batch	 19 	 Epoch:	 77
starting batch	 20 	 Epoch:	 77
starting batch	 21 	 Epoch:	 77
starting batch	 22 	 Epoch:	 77
starting batch	 23 	 Epoch:	 77
starting batch	 24 	 Epoch:	 77
starting batch	 25 	 Epoch:	 77
starting batch	 26 	 Epoch:	 77
starting batch	 27 	 Epoch:	 77
starting batch	 28 	 Epoch:	 77
starting batch	 29 	 Epoch:	 77
starting batch	 30 	 Epoch:	 77
starting batch	 31 	 Epoch:	 77
starting batch	 3

4/4 [==============================] - 0s 79ms/step - loss: 1.4080 - metric: 0.6719

Training accuracy: 0.901500	 Validation accuracy: 0.718750	 Testing Accuracy: 0.671875

Training loss: 0.905645    	 Validation loss: 1.309317    	 Testing Loss: 1.407999 


starting batch	 0 	 Epoch:	 82
starting batch	 1 	 Epoch:	 82
starting batch	 2 	 Epoch:	 82
starting batch	 3 	 Epoch:	 82
starting batch	 4 	 Epoch:	 82
starting batch	 5 	 Epoch:	 82
starting batch	 6 	 Epoch:	 82
starting batch	 7 	 Epoch:	 82
starting batch	 8 	 Epoch:	 82
starting batch	 9 	 Epoch:	 82
starting batch	 10 	 Epoch:	 82
starting batch	 11 	 Epoch:	 82
starting batch	 12 	 Epoch:	 82
starting batch	 13 	 Epoch:	 82
starting batch	 14 	 Epoch:	 82
starting batch	 15 	 Epoch:	 82
starting batch	 16 	 Epoch:	 82
starting batch	 17 	 Epoch:	 82
starting batch	 18 	 Epoch:	 82
starting batch	 19 	 Epoch:	 82
starting batch	 20 	 Epoch:	 82
starting batch	 21 	 Epoch:	 82
starting batch	 22 	 Epoch:	 82
starting batch	

starting batch	 13 	 Epoch:	 86
starting batch	 14 	 Epoch:	 86
starting batch	 15 	 Epoch:	 86
starting batch	 16 	 Epoch:	 86
starting batch	 17 	 Epoch:	 86
starting batch	 18 	 Epoch:	 86
starting batch	 19 	 Epoch:	 86
starting batch	 20 	 Epoch:	 86
starting batch	 21 	 Epoch:	 86
starting batch	 22 	 Epoch:	 86
starting batch	 23 	 Epoch:	 86
starting batch	 24 	 Epoch:	 86
starting batch	 25 	 Epoch:	 86
starting batch	 26 	 Epoch:	 86
starting batch	 27 	 Epoch:	 86
starting batch	 28 	 Epoch:	 86
starting batch	 29 	 Epoch:	 86
starting batch	 30 	 Epoch:	 86
starting batch	 31 	 Epoch:	 86
starting batch	 32 	 Epoch:	 86
starting batch	 33 	 Epoch:	 86
starting batch	 34 	 Epoch:	 86
starting batch	 35 	 Epoch:	 86
starting batch	 36 	 Epoch:	 86
starting batch	 37 	 Epoch:	 86
starting batch	 38 	 Epoch:	 86
starting batch	 39 	 Epoch:	 86
4/4 [==============================] - 0s 77ms/step - loss: 1.2481 - metric: 0.7812

Training accuracy: 0.922000	 Validation accuracy: 0

(0.8146103896103897, 0.77, 0.76170359052712, None)
starting batch	 0 	 Epoch:	 91
starting batch	 1 	 Epoch:	 91
starting batch	 2 	 Epoch:	 91
starting batch	 3 	 Epoch:	 91
starting batch	 4 	 Epoch:	 91
starting batch	 5 	 Epoch:	 91
starting batch	 6 	 Epoch:	 91
starting batch	 7 	 Epoch:	 91
starting batch	 8 	 Epoch:	 91
starting batch	 9 	 Epoch:	 91
starting batch	 10 	 Epoch:	 91
starting batch	 11 	 Epoch:	 91
starting batch	 12 	 Epoch:	 91
starting batch	 13 	 Epoch:	 91
starting batch	 14 	 Epoch:	 91
starting batch	 15 	 Epoch:	 91
starting batch	 16 	 Epoch:	 91
starting batch	 17 	 Epoch:	 91
starting batch	 18 	 Epoch:	 91
starting batch	 19 	 Epoch:	 91
starting batch	 20 	 Epoch:	 91
starting batch	 21 	 Epoch:	 91
starting batch	 22 	 Epoch:	 91
starting batch	 23 	 Epoch:	 91
starting batch	 24 	 Epoch:	 91
starting batch	 25 	 Epoch:	 91
starting batch	 26 	 Epoch:	 91
starting batch	 27 	 Epoch:	 91
starting batch	 28 	 Epoch:	 91
starting batch	 29 	 Epoch:	 91